In [11]:
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import seaborn as sns
from typing import List, Dict, Any, Tuple
import uuid
import random
from dataclasses import dataclass, field
import os
import warnings

class SemanticaPrimitive:
    """
    Represents a semantic primitive with rich contextual metadata
    """
    def __init__(self, 
                 value: str, 
                 primitive_type: str = 'generic', 
                 language: str = 'universal',
                 embedding: np.ndarray = None):
        self.id = str(uuid.uuid4())
        self.value = value
        self.type = primitive_type
        self.language = language
        self.embedding = embedding if embedding is not None else np.random.rand(50)
        self.metadata = {}
        self.semantic_energy = 0.5  # New attribute to represent semantic potency

    def __repr__(self):
        return f"Primitive(value={self.value}, type={self.type}, lang={self.language})"

@dataclass
class SemanticRelation:
    """
    Represents a relation between primitives with validation and confidence
    """
    source: SemanticaPrimitive
    target: SemanticaPrimitive
    relation_type: str
    confidence: float = 0.0
    agent_validations: List[str] = field(default_factory=list)
    semantic_intensity: float = 0.0
    
    def validate(self, agent_id: str):
        """Track agent validations and update confidence"""
        if agent_id not in self.agent_validations:
            self.agent_validations.append(agent_id)
            self.confidence = min(1.0, self.confidence + 0.1)
            self.semantic_intensity = len(self.agent_validations) * 0.2

class SemanticAgent:
    """
    An agent that can propose, validate, and modify semantic relations
    """
    def __init__(self, agent_id: str, expertise: Dict[str, float] = None):
        self.id = agent_id
        self.expertise = expertise or {}
        self.memory_graph = nx.DiGraph()

    def propose_relation(self, 
                         source: SemanticaPrimitive, 
                         target: SemanticaPrimitive, 
                         relation_type: str) -> SemanticRelation:
        """Propose a semantic relation based on agent's expertise"""
        confidence = self.expertise.get(relation_type, 0.5)
        relation = SemanticRelation(source, target, relation_type, confidence)
        relation.validate(self.id)
        
        # Boost semantic energy of primitives
        source.semantic_energy += 0.1
        target.semantic_energy += 0.1
        
        return relation

class SemanticaGraph:
    """
    Central semantic graph that manages primitives, relations, and convergence
    """
    def __init__(self, output_dir: str = None):
        self.primitives = {}
        self.relations = []
        self.agents = {}
        self.convergence_graph = nx.DiGraph()
        self.output_dir = output_dir or os.path.join(os.getcwd(), 'output')
        os.makedirs(self.output_dir, exist_ok=True)

    def add_primitive(self, primitive: SemanticaPrimitive):
        """Add a new primitive to the semantic space"""
        self.primitives[primitive.id] = primitive
        return primitive

    def add_agent(self, agent: SemanticAgent):
        """Register a new semantic agent"""
        self.agents[agent.id] = agent

    def propose_relation(self, 
                         source_primitive: SemanticaPrimitive, 
                         target_primitive: SemanticaPrimitive, 
                         relation_type: str,
                         proposing_agent: SemanticAgent):
        """Propose and potentially validate a semantic relation"""
        relation = proposing_agent.propose_relation(
            source_primitive, 
            target_primitive, 
            relation_type
        )
        
        # Cross-validate with other agents
        for agent in self.agents.values():
            if agent.id != proposing_agent.id:
                # More nuanced validation logic
                validation_prob = 0.7 * (1 + agent.expertise.get(relation_type, 0.5))
                if random.random() < validation_prob:
                    relation.validate(agent.id)
        
        self.relations.append(relation)
        
        # Update convergence graph
        self.convergence_graph.add_edge(
            source_primitive.id, 
            target_primitive.id, 
            relation_type=relation_type,
            confidence=relation.confidence,
            semantic_intensity=relation.semantic_intensity
        )

    def animate_convergence(self, num_frames: int = 100):
        """
        Create an animated visualization of semantic convergence
        """
        # Debugging: Check the state of primitives, relations, and graph
        print(f"Number of primitives: {len(self.primitives)}")
        print(f"Number of relations: {len(self.relations)}")
        print(f"Number of nodes in convergence graph: {self.convergence_graph.number_of_nodes()}")
        print(f"Number of edges in convergence graph: {self.convergence_graph.number_of_edges()}")

        fig, ax = plt.subplots(figsize=(15, 10), facecolor='black')
        plt.style.use('dark_background')

        def update(frame):
            ax.clear()
            ax.set_facecolor('black')
            plt.title("Semantic Convergence Dynamics", color='white', fontsize=16)

            # Periodically add new relations or validate existing ones
            if frame % 10 == 0:
                # Check if there are agents and primitives to work with
                if not self.agents:
                    warnings.warn("No agents available. Skipping agent-related logic.")
                    return
                if not self.primitives:
                    warnings.warn("No primitives available. Skipping primitive-related logic.")
                    return

                # Simulate new relation proposals or validations
                for _ in range(2):
                    source = random.choice(list(self.primitives.values()))
                    target = random.choice(list(self.primitives.values()))
                    agent = random.choice(list(self.agents.values()))
                    self.propose_relation(source, target, 'exploration', agent)

            # Prepare graph layout
            pos = nx.spring_layout(self.convergence_graph, k=0.5, iterations=50)

            # Node colors and sizes based on semantic energy
            node_colors = [self.primitives[node].semantic_energy for node in self.convergence_graph.nodes()]
            node_sizes = [100 + self.primitives[node].semantic_energy * 500 for node in self.convergence_graph.nodes()]

            # Edge weights and colors based on relation confidence
            edge_weights = [
                self.convergence_graph[u][v].get('confidence', 0.1) * 5 
                for (u, v) in self.convergence_graph.edges()
            ]
            edge_colors = [
                plt.cm.plasma(self.convergence_graph[u][v].get('confidence', 0.1)) 
                for (u, v) in self.convergence_graph.edges()
            ]

            # Draw nodes
            nx.draw_networkx_nodes(
                self.convergence_graph, 
                pos, 
                node_color=node_colors, 
                node_size=node_sizes,
                cmap=plt.cm.viridis,
                alpha=0.8
            )

            # Draw edges
            nx.draw_networkx_edges(
                self.convergence_graph, 
                pos, 
                width=edge_weights,
                edge_color=edge_colors,
                alpha=0.6,
                arrows=True,
                connectionstyle='arc3,rad=0.1'
            )

            # Draw labels
            nx.draw_networkx_labels(
                self.convergence_graph, 
                pos, 
                labels={node: self.primitives[node].value for node in self.convergence_graph.nodes()},
                font_size=8,
                font_color='white'
            )

            # Add frame number and global semantic energy
            total_semantic_energy = sum(p.semantic_energy for p in self.primitives.values())
            ax.text(0.02, 0.98, f"Frame: {frame} / {num_frames}", transform=ax.transAxes, color='white', fontsize=10, verticalalignment='top')
            ax.text(0.02, 0.93, f"Total Semantic Energy: {total_semantic_energy:.2f}", transform=ax.transAxes, color='white', fontsize=10, verticalalignment='top')

            plt.axis('off')

        # Create animation
        anim = animation.FuncAnimation(fig, update, frames=num_frames, interval=200)
        
        # Save the animation
        output_path = os.path.join(self.output_dir, f'semantic_convergence_{uuid.uuid4()}.gif')
        anim.save(output_path, writer='pillow', fps=5)
        plt.close(fig)
        
        print(f"Animation saved to {output_path}")
        return output_path

# Phase 1

In [ ]:

# Demonstration of Semantica principles with expanded vocabulary and relations
def semantic_convergence_demo(output_dir: str = None):
    # Initialize Semantica Graph
    semantica = SemanticaGraph(output_dir)
    
    # Create expanded primitives with richer context
    primitives = [
        # Fruits
        SemanticaPrimitive("apple", primitive_type="fruit", language="english"),
        SemanticaPrimitive("Apfel", primitive_type="fruit", language="german"),
        SemanticaPrimitive("manzana", primitive_type="fruit", language="spanish"),
        
        # Colors
        SemanticaPrimitive("red", primitive_type="color", language="english"),
        SemanticaPrimitive("rot", primitive_type="color", language="german"),
        SemanticaPrimitive("rojo", primitive_type="color", language="spanish"),
        
        # Tastes
        SemanticaPrimitive("sweet", primitive_type="taste", language="english"),
        SemanticaPrimitive("süß", primitive_type="taste", language="german"),
        SemanticaPrimitive("dulce", primitive_type="taste", language="spanish"),
        
        # Additional fruits
        SemanticaPrimitive("orange", primitive_type="fruit", language="english"),
        SemanticaPrimitive("Orange", primitive_type="fruit", language="german"),
        SemanticaPrimitive("naranja", primitive_type="fruit", language="spanish"),
        
        # Size attributes
        SemanticaPrimitive("big", primitive_type="size", language="english"),
        SemanticaPrimitive("small", primitive_type="size", language="english"),
        
        # Abstract concepts
        SemanticaPrimitive("health", primitive_type="concept", language="english"),
        SemanticaPrimitive("nutrition", primitive_type="concept", language="english"),
        
        # Food categories
        SemanticaPrimitive("fruit", primitive_type="category", language="english"),
        SemanticaPrimitive("dessert", primitive_type="category", language="english"),
        
        # Seasons
        SemanticaPrimitive("autumn", primitive_type="season", language="english"),
        SemanticaPrimitive("summer", primitive_type="season", language="english")
    ]
    
    # Add primitives to the graph
    primitive_dict = {}  # For easy reference later
    for p in primitives:
        semantica.add_primitive(p)
        primitive_dict[p.value] = p
    
    # Create agents with different expertise profiles
    agents = [
        SemanticAgent("linguist1", {"translation": 0.9, "similarity": 0.7, "exploration": 0.6, "categorization": 0.5}),
        SemanticAgent("linguist2", {"translation": 0.8, "similarity": 0.6, "exploration": 0.5, "categorization": 0.7}),
        SemanticAgent("translator", {"translation": 1.0, "similarity": 0.9, "exploration": 0.8, "categorization": 0.4}),
        SemanticAgent("nutritionist", {"health_relation": 0.9, "categorization": 0.8, "similarity": 0.5, "exploration": 0.7}),
        SemanticAgent("chef", {"taste_relation": 0.9, "similarity": 0.8, "categorization": 0.9, "exploration": 0.8})
    ]
    
    # Add agents to the graph
    for agent in agents:
        semantica.add_agent(agent)
    
    # Propose initial relations - translations between languages
    semantica.propose_relation(
        primitive_dict["apple"],
        primitive_dict["Apfel"],
        "translation",
        agents[0]
    )
    
    semantica.propose_relation(
        primitive_dict["apple"],
        primitive_dict["manzana"],
        "translation",
        agents[2]
    )
    
    semantica.propose_relation(
        primitive_dict["red"],
        primitive_dict["rot"],
        "translation",
        agents[1]
    )
    
    semantica.propose_relation(
        primitive_dict["red"],
        primitive_dict["rojo"],
        "translation",
        agents[2]
    )
    
    semantica.propose_relation(
        primitive_dict["sweet"],
        primitive_dict["süß"],
        "translation",
        agents[0]
    )
    
    semantica.propose_relation(
        primitive_dict["sweet"],
        primitive_dict["dulce"],
        "translation",
        agents[2]
    )
    
    # Descriptive relations - attributes of objects
    semantica.propose_relation(
        primitive_dict["apple"],
        primitive_dict["red"],
        "has_attribute",
        agents[0]
    )
    
    semantica.propose_relation(
        primitive_dict["apple"],
        primitive_dict["sweet"],
        "has_attribute",
        agents[4]  # chef
    )
    
    semantica.propose_relation(
        primitive_dict["orange"],
        primitive_dict["sweet"],
        "has_attribute",
        agents[4]  # chef
    )
    
    # Categorical relations
    semantica.propose_relation(
        primitive_dict["apple"],
        primitive_dict["fruit"],
        "is_a",
        agents[3]  # nutritionist
    )
    
    semantica.propose_relation(
        primitive_dict["orange"],
        primitive_dict["fruit"],
        "is_a",
        agents[3]  # nutritionist
    )
    
    semantica.propose_relation(
        primitive_dict["apple"],
        primitive_dict["dessert"],
        "can_be_used_as",
        agents[4]  # chef
    )
    
    # Conceptual relations
    semantica.propose_relation(
        primitive_dict["fruit"],
        primitive_dict["health"],
        "promotes",
        agents[3]  # nutritionist
    )
    
    semantica.propose_relation(
        primitive_dict["fruit"],
        primitive_dict["nutrition"],
        "provides",
        agents[3]  # nutritionist
    )
    
    # Seasonal relations
    semantica.propose_relation(
        primitive_dict["apple"],
        primitive_dict["autumn"],
        "associated_with",
        agents[0]
    )
    
    semantica.propose_relation(
        primitive_dict["orange"],
        primitive_dict["summer"],
        "associated_with",
        agents[0]
    )
    
    # Size relations
    semantica.propose_relation(
        primitive_dict["apple"],
        primitive_dict["small"],
        "has_typical_size",
        agents[1]
    )
    
    semantica.propose_relation(
        primitive_dict["orange"],
        primitive_dict["small"],
        "has_typical_size",
        agents[1]
    )
    
    # Create an animated visualization with more frames for complexity
    output_path = semantica.animate_convergence(num_frames=500)
    return semantica, output_path

# Run the demonstration
if __name__ == "__main__":
    # Specify the output directory
    output_dir = r"C:\Users\Erich Curtis\Desktop\All Python\Semantica\Data\Output"
    
    # Run the demo
    demo_graph, animation_path = semantic_convergence_demo(output_dir)
    print(f"Animation created at: {animation_path}")

# Phase 2

In [12]:
# Phase 2: Dynamic Data Integration and Enhanced Visualization
# This phase integrates ConceptNet datasets into the visualization pipeline, allowing dynamic sampling and ensuring at least 100 matching words between datasets.
import sys
sys.path.append(r'C:\Users\erich\OneDrive\Documents\Python Projects\Semantica\Semantica\Py Scripts')  # or the correct relative path from your notebook to Py Scripts

import pandas as pd
from conceptnet_processor import ConceptNetProcessor_v2

english_conceptnet = pd.read_csv('../Data/Input/conceptnet-assertions-5.7.0.en.tsv', sep='\t', names=['assertion', 'rel', 'start', 'end', 'meta'], header=None)
german_conceptnet = pd.read_csv('../Data/Input/conceptnet-assertions-5.7.0.de.tsv', sep='\t', names=['assertion', 'rel', 'start', 'end', 'meta'], header=None)

max_concepts = 500  # Maximum number of concepts to sample
# Load datasets
processor = ConceptNetProcessor_v2(english_data=english_conceptnet, german_data=german_conceptnet)
processor.build_semantic_graph(max_concepts=max_concepts, sample_size=0.05)

# After building the semantic graph
print(f'Number of nodes in the semantic graph: {len(processor.semantic_graph.nodes())}')
print(f'Sample nodes: {list(processor.semantic_graph.nodes())[:10]}')

# Check the intersection of nodes
matching_words = set(processor.semantic_graph.nodes()) & set(processor.semantic_graph.nodes())
print(f'Number of matching words: {len(matching_words)}')
print(f'Sample matching words: {list(matching_words)[:10]}')

ConceptNetProcessor initialized
Building semantic graph from ConceptNet data...
Processing 3423004 English ConceptNet assertions...
Will sample 171150 English assertions
Processing 1078946 German ConceptNet assertions...
Will sample 53947 German assertions
Sampling 171150 assertions from 3423004 en assertions


Processing en assertions: 100%|██████████| 171150/171150 [00:05<00:00, 29425.31it/s]


Sampling 53947 assertions from 1078946 de assertions


Processing de assertions: 100%|██████████| 53947/53947 [00:02<00:00, 25391.29it/s]


Limiting graph to top 500 concepts...
Semantic graph built with 500 nodes and 2519 edges
Inferring semantic categories...
Inferred categories: {'generic': 447, 'place': 48, 'person': 5}
Number of nodes in the semantic graph: 500
Sample nodes: ['philippines', 'service', 'area', 'genetics', 'mining', 'small', 'plane', 'italy', 'cognition', 'ship']
Number of matching words: 500
Sample matching words: ['philippines', 'service', 'area', 'genetics', 'mining', 'small', 'plane', 'italy', 'cognition', 'disorder']


In [13]:
# Ensure at least 100 matching words
matching_words = set(processor.semantic_graph.nodes()) & set(processor.semantic_graph.nodes())
if len(matching_words) < max_concepts:
    raise ValueError("Not enough matching words between datasets.")

In [ ]:
output_dir = r"C:\Users\erich\OneDrive\Documents\Python Projects\Semantica\Semantica\Data\Output"
# Update primitives and relations dynamically
semantica = SemanticaGraph(output_dir)

# Create a dictionary to store primitives for easy access
primitive_dict = {}

# Add primitives from ConceptNet
for word in matching_words:
    primitive = SemanticaPrimitive(value=word)
    semantica.add_primitive(primitive)
    primitive_dict[word] = primitive

# Create agents to work with the ConceptNet data
agents = [
    SemanticAgent("conceptnet_expert", {"translation": 0.9, "similarity": 0.8, "categorization": 0.9, "exploration": 0.7}),
    SemanticAgent("linguistic_analyst", {"similarity": 0.9, "exploration": 0.8, "categorization": 0.7}),
    SemanticAgent("knowledge_integrator", {"categorization": 0.9, "exploration": 0.9}),
    SemanticAgent("semantic_explorer", {"exploration": 1.0, "similarity": 0.7})
]

# Add agents to the semantica graph
for agent in agents:
    semantica.add_agent(agent)

# Extract meaningful relations from the ConceptNet processor's semantic graph
print("Adding relations based on ConceptNet data...")
relation_count = 0
relation_types = {"IsA", "RelatedTo", "AtLocation", "HasProperty", "UsedFor", "CapableOf", "HasA", "PartOf"}

# Add relations based on the processor's semantic graph edges
for source, target, data in processor.semantic_graph.edges(data=True):
    if source in primitive_dict and target in primitive_dict:
        rel_type = data.get('relation', 'RelatedTo')
        rel_short = rel_type.split('/')[-1] if '/' in rel_type else rel_type
        
        # Choose an appropriate agent based on relation type
        if rel_short in ["IsA", "PartOf"]:
            chosen_agent = agents[2]  # knowledge_integrator for categorization
        elif rel_short in ["RelatedTo", "SimilarTo"]:
            chosen_agent = agents[1]  # linguistic_analyst for similarity
        elif rel_short in ["HasProperty", "HasA"]:
            chosen_agent = agents[0]  # conceptnet_expert for properties
        else:
            chosen_agent = agents[3]  # semantic_explorer for other relations
        
        # Propose the relation
        semantica.propose_relation(
            primitive_dict[source],
            primitive_dict[target],
            rel_short,
            chosen_agent
        )
        relation_count += 1
        
        # Limit the number of relations to avoid overwhelming visualization
        if relation_count >= 300:
            break

# Add some additional cross-relations to enhance connectivity
print(f"Added {relation_count} relations from ConceptNet data")

# Sample some common concepts to create additional relations
common_concepts = [word for word in matching_words if len(word) < 10][:50]
for i in range(30):  # Add 30 additional exploration relations
    if len(common_concepts) >= 2:
        source_word = random.choice(common_concepts)
        target_word = random.choice(common_concepts)
        if source_word != target_word and source_word in primitive_dict and target_word in primitive_dict:
            semantica.propose_relation(
                primitive_dict[source_word],
                primitive_dict[target_word],
                "exploration",
                agents[3]
            )

# Print statistics about the graph we've built
print(f"Created graph with {len(semantica.primitives)} primitives and {len(semantica.relations)} relations")
print(f"Convergence graph has {semantica.convergence_graph.number_of_nodes()} nodes and {semantica.convergence_graph.number_of_edges()} edges")

# Create enhanced animation
output_path = semantica.animate_convergence(num_frames=300)
print(f"Enhanced animation created at: {output_path}")

Adding relations based on ConceptNet data...
Added 300 relations from ConceptNet data
Created graph with 500 primitives and 330 relations
Convergence graph has 303 nodes and 330 edges
Number of primitives: 500
Number of relations: 330
Number of nodes in convergence graph: 303
Number of edges in convergence graph: 330
